## Draft notebook to check the performance of the functions

Creating a json with encoded images from the photos folder

In [1]:
from base64 import b64encode
from json import dumps
import os

ENCODING = 'utf-8'
FOLDER_NAME = 'photos'
JSON_NAME = 'output.json'
tree = {FOLDER_NAME:[]}

image_names = [f for f in os.listdir(FOLDER_NAME) if f.endswith('.jpg')]
for img_name in image_names:
    with open(FOLDER_NAME+'/'+img_name, 'rb') as open_file:
        byte_content = open_file.read()
    base64_bytes = b64encode(byte_content)
    base64_string = base64_bytes.decode(ENCODING)
    raw_data = {"ID": img_name,"img_code": base64_string}
    tree[FOLDER_NAME].append(raw_data)
json_data = dumps(tree, indent=2)
with open(JSON_NAME, 'w') as another_open_file:
    another_open_file.write(json_data)

Checking whether image is encoded

In [2]:
import json

with open('output.json') as f:
  data = json.load(f)
print(data["photos"][0:5])

[{'ID': 'cat.0.jpg', 'img_code': '/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAoHBwgHBgoICAgLCgoLDhgQDg0NDh0VFhEYIx8lJCIfIiEmKzcvJik0KSEiMEExNDk7Pj4+JS5ESUM8SDc9Pjv/2wBDAQoLCw4NDhwQEBw7KCIoOzs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozs7Ozv/wAARCAF2AfQDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwDokqVKiSpVr5s9UlWpFqNalWkMeop4pFIqQY9KAAU4UDbS/LSAUU4GmilFADxTxUamng0CY8Y6Gq1zb5BOKsCn4BGDQJOzOZvbIPkFQaz49PW3bkV1U9sM5ArOuYO9aRfc6ITM7Y

Decode the image from base64 string

In [4]:
import base64

#os.chdir(os.path.dirname(os.path.abspath(__file__)))
dir_decoded = 'images'
if not os.path.isdir(dir_decoded):
    os.mkdir(dir_decoded)
for img_dict in data["photos"]:
    encoded_img = img_dict["img_code"]
    decoded_img = open(dir_decoded+'/'+img_dict["ID"],'wb')
    decoded_img.write(base64.b64decode((encoded_img)))
    decoded_img.close()

Check the performance of the chosen classification model

In [43]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model

new_modell = load_model('model.h5')
# new_model.summary()
filename = 'cat.1.jpg'
test_image = image.load_img('images/' + filename, target_size=(64, 64))
# test_image = image.load_img("/content/sample_data/train/cat.1041.jpg", target_size=(128, 128))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = new_modell.predict(test_image)
ans = np.argmax(result, axis=1)

if ans==1:
    prediction="dog"
else:
    prediction = "cat"

print("Result",result)
print("Answer",ans)
print("Prediction", prediction)

Result [[0.]]
Prediction cat


Applying model predictions to all images

In [69]:
from base64 import b64encode
from json import dumps
import os
import pandas as pd

ENCODING = 'utf-8'
FOLDER_NAME = 'images'
JSON_NAME = 'results.json'
results = {"Results":[]}
counter = 1
new_model = load_model('model.h5')
df = pd.DataFrame()
preds = []
trues = []
dogs = []
cats = []


image_names = [f for f in os.listdir(FOLDER_NAME) if f.endswith('.jpg')]
for img_name in image_names:
    test_image = image.load_img(FOLDER_NAME+'/'+img_name, target_size=(64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis=0)
    result = new_model.predict(test_image)
    dog_prob = result[0][1]
    cat_prob = result[0][0]
    if cat_prob > dog_prob:
        prediction = 'cat'
    else:
        prediction = 'dog'
    
    preds.append(prediction)
    trues.append(img_name.split('.')[0])
    dogs.append(dog_prob)
    cats.append(cat_prob)
    store_data = {"ID": img_name,"cat_prob": str(np.round(cat_prob,3)),"dog_prob":str(np.round(dog_prob,3))}
    
    results["Results"].append(store_data)
    print(str(counter),'/',str(len(image_names)))
    counter+=1
json_results = dumps(results, indent=2)
with open(JSON_NAME, 'w') as results_file:
    results_file.write(json_results)



1 / 1000
2 / 1000
3 / 1000
4 / 1000
5 / 1000
6 / 1000
7 / 1000
8 / 1000
9 / 1000
10 / 1000
11 / 1000
12 / 1000
13 / 1000
14 / 1000
15 / 1000
16 / 1000
17 / 1000
18 / 1000
19 / 1000
20 / 1000
21 / 1000
22 / 1000
23 / 1000
24 / 1000
25 / 1000
26 / 1000
27 / 1000
28 / 1000
29 / 1000
30 / 1000
31 / 1000
32 / 1000
33 / 1000
34 / 1000
35 / 1000
36 / 1000
37 / 1000
38 / 1000
39 / 1000
40 / 1000
41 / 1000
42 / 1000
43 / 1000
44 / 1000
45 / 1000
46 / 1000
47 / 1000
48 / 1000
49 / 1000
50 / 1000
51 / 1000
52 / 1000
53 / 1000
54 / 1000
55 / 1000
56 / 1000
57 / 1000
58 / 1000
59 / 1000
60 / 1000
61 / 1000
62 / 1000
63 / 1000
64 / 1000
65 / 1000
66 / 1000
67 / 1000
68 / 1000
69 / 1000
70 / 1000
71 / 1000
72 / 1000
73 / 1000
74 / 1000
75 / 1000
76 / 1000
77 / 1000
78 / 1000
79 / 1000
80 / 1000
81 / 1000
82 / 1000
83 / 1000
84 / 1000
85 / 1000
86 / 1000
87 / 1000
88 / 1000
89 / 1000
90 / 1000
91 / 1000
92 / 1000
93 / 1000
94 / 1000
95 / 1000
96 / 1000
97 / 1000
98 / 1000
99 / 1000
100 / 1000
101 / 10

756 / 1000
757 / 1000
758 / 1000
759 / 1000
760 / 1000
761 / 1000
762 / 1000
763 / 1000
764 / 1000
765 / 1000
766 / 1000
767 / 1000
768 / 1000
769 / 1000
770 / 1000
771 / 1000
772 / 1000
773 / 1000
774 / 1000
775 / 1000
776 / 1000
777 / 1000
778 / 1000
779 / 1000
780 / 1000
781 / 1000
782 / 1000
783 / 1000
784 / 1000
785 / 1000
786 / 1000
787 / 1000
788 / 1000
789 / 1000
790 / 1000
791 / 1000
792 / 1000
793 / 1000
794 / 1000
795 / 1000
796 / 1000
797 / 1000
798 / 1000
799 / 1000
800 / 1000
801 / 1000
802 / 1000
803 / 1000
804 / 1000
805 / 1000
806 / 1000
807 / 1000
808 / 1000
809 / 1000
810 / 1000
811 / 1000
812 / 1000
813 / 1000
814 / 1000
815 / 1000
816 / 1000
817 / 1000
818 / 1000
819 / 1000
820 / 1000
821 / 1000
822 / 1000
823 / 1000
824 / 1000
825 / 1000
826 / 1000
827 / 1000
828 / 1000
829 / 1000
830 / 1000
831 / 1000
832 / 1000
833 / 1000
834 / 1000
835 / 1000
836 / 1000
837 / 1000
838 / 1000
839 / 1000
840 / 1000
841 / 1000
842 / 1000
843 / 1000
844 / 1000
845 / 1000
846 / 1000

Creating a pandas dataframe with model performance

In [65]:
df["ID"] = image_names
df["Predictions"] = preds
df["Trues"] = trues
true_bin = [1 if i==j else 0 for i,j in zip(preds,trues)]
df["Dog_prob"] = dogs
df["Cat_prob"] = cats
df['True_bin'] = true_bin

df

,ID,Predictions,Trues,Dog_prob,Cat_prob,True_bin
0,cat.0.jpg,cat,cat,0.0,1.0,1
1,cat.1.jpg,cat,cat,0.0,1.0,1
2,cat.10.jpg,cat,cat,0.0,1.0,1
3,cat.100.jpg,dog,cat,1.0,0.0,0
4,cat.101.jpg,dog,cat,1.0,0.0,0
...,...,...,...,...,...,...
995,dog.95.jpg,dog,dog,1.0,0.0,1
996,dog.96.jpg,dog,dog,1.0,0.0,1
997,dog.97.jpg,dog,dog,1.0,0.0,1
998,dog.98.jpg,dog,dog,1.0,0.0,1


Calculating model accuracy

In [66]:
accuracy = sum(df["True_bin"]) / len(df["True_bin"])
print(accuracy)

0.842
